### **Importing required packages:**

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

import pygsm
from pygsm import GlobalSkyModel

import ephem
from astropy.time import Time
from astropy.coordinates import Galactic, SkyCoord, FK5, EarthLocation, AltAz, Angle
import astropy.units as u

import pandas as pd
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline

### **Defining some constants:**

In [ ]:
h = 6.62607157e-34
k = 1.3806488e-23
c = 2.99792458e+08
TCMB = 2.72548


### **Sky Model part code below:** 

In [ ]:
NPIX = hp.nside2npix(16)
T = np.empty(NPIX)

gsm = GlobalSkyModel(freq_unit='GHz')

#**Generating maps for frequencies from 2 to 4 GHz in increment of 100MHz:**

freqs = np.linspace(2,4, 21)
CMB_Int = np.empty(21)
P_hot = np.empty(21)
P_cold = np.empty(21)
P_dif = np.empty(21)
spectrum = gsm.generate(freqs)

#Decreasing the resolution to NSIDE = 16

spec = hp.pixelfunc.ud_grade(spectrum,16)
df = pd.DataFrame(spec)
df = df.transpose()
df.columns = np.arange(2,4.1,0.1)
df = df*1e-20
#Basically 1e-20 factor is to convert MJy/Sr (PyGSM pack has this units) to W/m^2 Hz (SI units)



#Calculation of CMB intensity below for each frequencies

i=0
while i<21:
    #Calculation of CMB intensities at different frequencies below
    CMB_Int[i] = (2*h*math.pow(df.columns[i]*1e9,3.0)/(c*c*math.exp(h*df.columns[i]*1e9/k*TCMB)-1))
    #P_hot and P_cold calculation for calibration below
    P_hot[i] = (df.columns[i]*373)   #Dummy function. Have to look the function
    P_cold[i] = (df.columns[i]*273)    #Dummy function. Haven't looked the function still
    P_dif[i] = P_hot[i] - P_cold[i]      #Dummy... its obvious now!
    
    i=i+1
    

CMB = pd.DataFrame(CMB_Int)
CMB = CMB.transpose()
CMB.columns = np.arange(2,4.1,0.1)
CMB = CMB.transpose()
df = df.transpose()

CMB_plus_Gal = np.add(df, CMB.to_numpy())    #Adding CMB and Galactic contribution intensities in SI units. Recombination line intensity is added later.
print(CMB_plus_Gal)
print(P_dif)







### **LST calculation and Coordinate transform:**

In [ ]:
#Sidereal time calculation

Obs = ephem.Observer()
t = Time('2022-05-09 16:15:00.0000', scale='utc',location=('10', '0'))
lst = t.sidereal_time('apparent')
print(lst)


#Checking the number of pixels

NPIX = hp.nside2npix(16)
print(NPIX)

i=1
alt = np.empty(NPIX)
az = np.empty(NPIX)

#For every pixel, get the horizontal coordinate. 

while i<NPIX:
    coords = hp.pixelfunc.pix2ang(16, i , nest = False, lonlat = True)
    gc = SkyCoord(l = coords[0]*u.degree, b= coords[1]*u.degree, frame = 'galactic')
    
    equi = gc.transform_to(AltAz(obstime = '2022-06-08T11:10:10.10', location = EarthLocation(lat = 12.9716*u.deg, lon=77.5946*u.deg, height=900*u.m)))
    
    #Getting the magnitude of azimuthal value alone in deg
    az[i-1]= equi.az .value 
    
    #Getting the magnitude of altitude value alone in deg
    alt[i-1] = equi.alt .value 
    
    i=i+1

print(az) #List of az for each pixel as an array
print(alt) #List of alt for each pixel as an array


### **Recombination data: Intensity vs frequency**

In [ ]:
df = pd.read_csv(r'/home/dhashin/Downloads/total_spec_new.txt',sep= "    " ,header = None)


df.reset_index(drop=True, inplace=True)
df.columns = ["Frequency","Intensity"]
print(df)

#ploting graph of intensity vs frequency

plt.plot(df["Frequency"],df["Intensity"])
plt.xlim(10,5000)
plt.xlabel('Frequency(GHz)')
plt.ylabel('Intensity(W/(Hzm^2)')

plt.show()

x = df["Frequency"]
y= df["Intensity"]

ius = InterpolatedUnivariateSpline(x, y)

y = ius(np.arange(2,4.1,0.1))
Rec_spec = pd.DataFrame(y)
print(Rec_spec)     #Getting intensities at the required frequencies

Total_sky = np.add(CMB_plus_Gal, Rec_spec.to_numpy())     #Adding reco. signal with gala. signal and CMB (All in SI units)
print(Total_sky)

P_dif = pd.DataFrame(P_dif)
print(P_dif)

T_sky = 100* np.divide(Total_sky,P_dif)   #100K is the difference of T_hot and T_cold. 
print(T_sky)




### **Function to generate map of visible sky given the location and time:**

In [ ]:
NPIX = hp.nside2npix(16)
print(NPIX)

gsm = GlobalSkyModel(freq_unit='GHz')


def Visible_Sky(frequency, latitude, longitude, altitude, T):           #Here frequency is taken in GHz, lat and lon in degrees and T is the format 'YYYY-MM-DD HH:MM:SS.SS'
    spectrum = gsm.generate(frequency)
    spectrum = hp.pixelfunc.ud_grade(spectrum, 16)                      #Reducing the resolution to NSIDE=16

    i=1

    while i<NPIX:           #Working on each pixel
        coords = hp.pixelfunc.pix2ang(16, i , nest = False, lonlat = True)          #Gives l and b coordinates for each pixel
        gc = SkyCoord(l = coords[0]*u.degree, b= coords[1]*u.degree, frame = 'galactic')
        
        #Now, the coordinates in Galactic frame is changed to equitorial frame below
        equi = gc.transform_to(AltAz(obstime = T, location = EarthLocation(lat = latitude*u.deg, lon=longitude*u.deg, height=altitude*u.m)))
        
        #If alt<0 in horizontal coordinates, that is not visible for the observer. So, suxh pixels are given minimum value (grey in map)
        if (equi.alt<0):
            spectrum[i-1] = -1.6375e+30
    
        i = i+1  
    print(spectrum)
    
spectrum_value = Visible_Sky(2,10,10,200,'2022-05-15 12:00:00')
print(spectrum_value)


In [ ]:
#Changing intensity to temperature scale

# for cols in df:
#     df[cols] = df[cols].apply(lambda x:(h* df.columns*1e9)/(k) * (0.5 + (x*1.25664e-20*c*c/(2*h*df.columns*df.columns*df.columns*1e27))))


### **Calculating az, alt for each pixels and adding refraction correction:**

In [ ]:
latitude = 12.9716
longitude = 77.5946
altitude = 900
T = '2022-05-22 12:00:00.00'

def refraction(a, b):
    return 1         #Dummy function just to check for errors

i=1
while i<NPIX:           #Working on each pixel
    coords = hp.pixelfunc.pix2ang(16, i , nest = False, lonlat = True)          #Gives l and b coordinates for each pixel
    gc = SkyCoord(l = coords[0]*u.degree, b= coords[1]*u.degree, frame = 'galactic')
        
        #Now, the coordinates in Galactic frame is changed to equitorial frame below
    equi = gc.transform_to(AltAz(obstime = T, location = EarthLocation(lat = latitude*u.deg, lon=longitude*u.deg, height=altitude*u.m)))
    alt = refraction(equi.alt, altitude)
    
    print(alt)
    i=i+1
